In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("test_lib_from_postgres_to_clickhouse")
    .config(
        "spark.jars.packages",
        ",".join([
            # clickhouse drivers
            "com.clickhouse:clickhouse-jdbc:0.7.2"
            # postgres drivers
            "org.postgresql:postgresql:42.7.3"
        ])
    )
    .getOrCreate()
)

In [ ]:
from data_transfer_lib import Postgres, ClickHouse, Reader, Writer

# Создание коннектов
postgres_conn = Postgres(
    host="postgres",
    user="postgres_user",
    password="postgres_password",
    database="postgres",
    spark=spark
)

clickhouse_conn = ClickHouse(
    host="clickhouse-server",
    user="default",
    password="1234qwe",
    database="default",
    spark=spark
)

# Чтение из Postgres
reader = Reader(
    connection=postgres_conn,
    db_name="mydb",
    table_name="users"
)
df = reader.start()

# Загрузка в ClickHouse
uploader = Writer(
    connection=clickhouse_conn,
    db_name="analytics",
    table_name="users",
    if_exists=True
)
uploader.start(df=df)

Вызван метод _get_or_create_spark
Инициализирован Postgres
Вызван метод Postgres._validate_connection_params
Вызван метод _get_or_create_spark
Инициализирован ClickHouse
Вызван метод ClickHouse._validate_connection_params
Инициализирован Reader
Вызван метод Reader._prepare
Вызван метод Postgres.get_table_schema для mydb.users
Вызван метод SchemaValidator.validate_source_to_spark
Вызван метод Reader.start для mydb.users
Параметры: {}
Инициализирован Writer
Вызван метод Writer._prepare
Вызван метод ClickHouse.get_table_schema для analytics.users
Получена схема target таблицы analytics.users
Вызван метод Writer.start для analytics.users
Параметры: {}
Вызван метод SchemaValidator.validate_spark_to_target
Данные успешно загружены (заглушка)
